# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

demographic-person(__id__, gender, age)
 
medications_drugs(__code__, name)
 
medications_use(__person_id__, __drug_code__, days_use)

Obs: person_id é uma chave estrangeira que se refere a id da tabela demographic-person e drug_code é uma chave estrageira que se refere a code da tabela medications_use.

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [10]:
%defaultDatasource jdbc:h2:mem:db

In [11]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Remedio;
DROP TABLE IF EXISTS UsoRemedio;


In [12]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

In [13]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [14]:
CREATE TABLE Remedio(
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(60),
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

In [15]:
SELECT * FROM Remedio;

In [16]:
CREATE TABLE UsoRemedio(
  Person_id VARCHAR(5) NOT NULL,
  Drug_code VARCHAR(6) NOT NULL,
  Days_use  INTEGER,
  FOREIGN KEY(Person_id)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION 
      ON UPDATE NO ACTION,
  FOREIGN KEY(Drug_code)
    REFERENCES Remedio(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [17]:
SELECT * FROM UsoRemedio;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [18]:
SELECT Id FROM Pessoa WHERE Age >= 60;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [19]:
SELECT DISTINCT Age FROM Pessoa;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [20]:
SELECT Pessoa.Id, Pessoa.Age, UsoRemedio.Drug_code, UsoRemedio.Days_use FROM Pessoa, UsoRemedio WHERE Pessoa.Id = UsoRemedio.Person_id;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [21]:
SELECT Pessoa.Id, Pessoa.Age,Remedio.Name, UsoRemedio.Days_use FROM Pessoa, Remedio, UsoRemedio WHERE Pessoa.Id = UsoRemedio.Person_id AND Remedio.Code = UsoRemedio.Drug_code;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [23]:
SELECT DISTINCT Remedio.Name FROM Remedio,UsoRemedio WHERE Remedio.Code = UsoRemedio.Drug_code;

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [24]:
SELECT DISTINCT Remedio.Name FROM Remedio,UsoRemedio WHERE Remedio.Code = UsoRemedio.Drug_code AND UsoRemedio.Days_use > 2000;